In [1]:

%pprint

Pretty printing has been turned OFF


In [132]:

from bs4.element import NavigableString
import random
import re
import numpy as np
import os
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

%run ../load_magic/storage.py
s = Storage()

lt_regex = re.compile(r'\s+<')
gt_regex = re.compile(r'>\s+')
saves_html_folder = os.path.join(s.saves_folder, 'html')
basic_tags_dict = s.load_object('basic_tags_dict')

def clean_html_str(html_obj):
    html_str = str(html_obj)
    html_str = html_str.strip()
    html_str = lt_regex.sub('<', html_str)
    html_str = gt_regex.sub('>', html_str)
    
    return html_str

def get_navigable_children(tag, result_list=[]):
    if type(tag) is not NavigableString:
        for child_tag in tag.children:
            result_list = get_navigable_children(child_tag, result_list)
    else:
        base_str = clean_html_str(tag)
        if base_str:
            tag_str = clean_html_str(tag.parent)
            if tag_str.count('<') > 2:
                tag_str = base_str
            result_list.append(tag_str)
    
    return result_list

def update_child_strs_list_dict(child_strs_list_dict=None, basic_tags_dict=None):
    if basic_tags_dict is None:
        basic_tags_dict = s.load_object('basic_tags_dict')
    if child_strs_list_dict is None:
        files_list = os.listdir(saves_html_folder)
        child_strs_list_dict = {}
        for file_name in files_list:
            file_path = os.path.join(saves_html_folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                html_str = f.read()
                job_soup = BeautifulSoup(html_str, 'lxml')
                body_soup = job_soup.find_all(name='body')[0]
                child_strs_list = get_navigable_children(body_soup, [])
            if not len(child_strs_list):
                os.remove(file_path)
            child_strs_list = [child_str for child_str in child_strs_list if child_str not in basic_tags_dict]
            child_strs_list_dict[file_name] = child_strs_list
        
        return child_strs_list_dict
    
    for file_name, child_strs_list in child_strs_list_dict.items():
        child_strs_list = [child_str for child_str in child_strs_list if child_str not in basic_tags_dict]
        if not len(child_strs_list):
            child_strs_list = child_strs_list_dict.pop(file_name)
            break
        else:
            child_strs_list_dict[file_name] = child_strs_list
    
    return child_strs_list_dict

In [73]:

child_strs_list_dict = update_child_strs_list_dict()


----

In [183]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

scanner_regex = re.compile(r'(</?|\b)[1-9a-zA-Z][0-9a-zA-Z]*( *[#\+]{1,2}|>|:\b|\.\b|\b)')
try:
    fit_estimators_dict = s.load_object('fit_estimators_dict')
except:
    fit_estimators_dict = {}
    fit_estimators_dict['LogisticRegression'] = LogisticRegression(**{'C': 7.5, 'class_weight': 'balanced', 'dual': True, 'fit_intercept': False,
                                                                      'max_iter': 64, 'penalty': 'l2', 'solver': 'liblinear', 'tol': 1e-09})
    s.store_objects(fit_estimators_dict=fit_estimators_dict)
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]

In [198]:

basic_tags_dict = s.load_object('basic_tags_dict')
child_strs_list_dict = update_child_strs_list_dict(child_strs_list_dict, basic_tags_dict)
for file_name, child_strs_list in child_strs_list_dict.items():
    for tag_str in child_strs_list:
        print()
        print()
        if "'" in tag_str:
            print(f'''
basic_tags_dict["{tag_str}"] = False
s.store_objects(basic_tags_dict=basic_tags_dict)''')
        else:
            print(f'''
basic_tags_dict['{tag_str}'] = False
s.store_objects(basic_tags_dict=basic_tags_dict)''')
        break
    break




basic_tags_dict['<p>Benefits:</p>'] = False
s.store_objects(basic_tags_dict=basic_tags_dict)


In [199]:

basic_tags_dict['<p>Benefits:</p>'] = True
s.store_objects(basic_tags_dict=basic_tags_dict)

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\basic_tags_dict.pickle


In [ ]:

for child_str, pred_array in zip(child_strs_list, predict_percent_fit(child_strs_list)):
    print()
    print(pred_array)
    print(child_str)

In [200]:

# Re-transform the bag-of-words and tf-idf from the new manual scores
basic_tags_dict = s.load_object('basic_tags_dict')
rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in basic_tags_dict.items()]
child_str_df = pd.DataFrame(rows_list)

if child_str_df.shape[0]:
    sents_list = child_str_df.navigable_parent.tolist()
    
    # Bag-of-words
    cv = CountVectorizer(lowercase=False, tokenizer=regex_tokenizer, ngram_range=(1, 3))
    bow_matrix = cv.fit_transform(sents_list)
    s.store_objects(cs_cv_vocab=cv.vocabulary_)
    
    # Tf-idf must get from Bag-of-words first
    tt = TfidfTransformer()
    tfidf_matrix = tt.fit_transform(bow_matrix)
    s.store_objects(cs_tt=tt)
    
    # Re-train the classifier
    X = tfidf_matrix.toarray()
    y = child_str_df.is_header.to_numpy()
    fit_estimators_dict = s.load_object('fit_estimators_dict')
    child_str_clf = fit_estimators_dict['LogisticRegression']
    child_str_clf.fit(X, y)
    fit_estimators_dict['LogisticRegression'] = child_str_clf
    s.store_objects(child_str_clf=child_str_clf, fit_estimators_dict=fit_estimators_dict)
    
    # Re-calibrate the inference engine
    cs_cv_vocab = s.load_object('cs_cv_vocab')
    cs_cv = CountVectorizer(vocabulary=cs_cv_vocab)
    cs_cv._validate_vocabulary()
    cs_tt = s.load_object('cs_tt')
    def predict_percent_fit(navigable_parents_list):
        y_predict_proba = np.array([])
        if len(navigable_parents_list):
            X_test = cs_tt.transform(cs_cv.transform(navigable_parents_list)).toarray()
            y_predict_proba = child_str_clf.predict_proba(X_test)

        return y_predict_proba
print('Retraining complete')

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\cs_cv_vocab.pickle
Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\cs_tt.pickle
Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\child_str_clf.pickle
Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\fit_estimators_dict.pickle
Retraining complete


In [201]:

sample_list = []
try:
    cs_cv_vocab = s.load_object('cs_cv_vocab')
    sample_list = random.sample([(w, i) for w, i in cs_cv_vocab.items()], 20)
except:
    cs_cv_vocab = {}
    s.store_objects(cs_cv_vocab=cs_cv_vocab)
sorted([(w, round(cs_tt.idf_[i], 2)) for w, i in sample_list], key=lambda x: x[1], reverse=False)

[('way', 4.33), ('development of causal', 4.74), ('The Statistician is', 4.74), ('Full Time', 4.74), ('team This person', 4.74), ('Keep the', 4.74), ('contributed content millions', 4.74), ('<p> Job', 4.74), ('Plus </b>', 4.74), ('people', 4.74), ('closely tied', 4.74), ('as possible We', 4.74), ('their analytical statistical', 4.74), ('build predictive', 4.74), ('application', 4.74), ('ASAP', 4.74), ('Yelp we believe', 4.74), ('those', 4.74), ('or disability', 4.74), ('environment </div>', 4.74)]

In [168]:

[fn for fn in dir(cv) if not fn.startswith('_')]

['analyzer', 'binary', 'build_analyzer', 'build_preprocessor', 'build_tokenizer', 'decode', 'decode_error', 'dtype', 'encoding', 'fit', 'fit_transform', 'fixed_vocabulary_', 'get_feature_names', 'get_params', 'get_stop_words', 'input', 'inverse_transform', 'lowercase', 'max_df', 'max_features', 'min_df', 'ngram_range', 'preprocessor', 'set_params', 'stop_words', 'stop_words_', 'strip_accents', 'token_pattern', 'tokenizer', 'transform', 'vocabulary', 'vocabulary_']

In [173]:

random.sample([w for w in cv.get_feature_names()], 20)

['quantitative', 'We re', 'diverse backgrounds and', '<b> Skills', 'degree', 'Media', 'hugely impactful', 'join', 'growing as', 'Applied scientists uncover', 'a new', 'Authenticity </i>', '401', 'problem solver with', 'expect you', 'believe that diversity', 'Clients for', 'quantitative discipline', 'we will consider', 'accept']

In [176]:

cv.tokenizer??

Signature: cv.tokenizer(corpus)
Docstring: <no docstring>
Source:   
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]
File:      c:\users\dev\documents\repositories\job-hunting\ipynb\<ipython-input-23-644ac792dd2c>
Type:      function



----

In [6]:

if len(child_strs_list):
    df = pd.DataFrame(child_strs_list, columns=['navigable_parent'])
    df['is_header'] = False
    id_list = []
    df.loc[id_list, 'is_header'] = True
    df.tail(60)

In [7]:

try:
    for row_index, row_series in df.iterrows():
        navigable_parent = row_series.navigable_parent
        is_header = row_series.is_header
        basic_tags_dict[navigable_parent] = is_header
    s.store_objects(basic_tags_dict=basic_tags_dict)
    rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in basic_tags_dict.items()]
    child_str_df = pd.DataFrame(rows_list)
    s.store_objects(child_str_df=child_str_df)
except Exception as e:
    print(str(e).strip())

name 'df' is not defined



----

In [14]:

try:
    basic_tags_dict = s.load_object('basic_tags_dict')
    rows_list = [{'navigable_parent': navigable_parent, 'is_header': is_header} for navigable_parent, is_header in basic_tags_dict.items()]
    child_str_df = pd.DataFrame(rows_list)
    s.store_objects(child_str_df=child_str_df)
except:
    child_str_df = s.load_object('child_str_df')
    try:
        child_str_df = pd.concat([child_str_df, df])
    except:
        pass
    basic_tags_dict = child_str_df.set_index('navigable_parent').to_dict()['is_header']
    s.store_objects(basic_tags_dict=basic_tags_dict)

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\child_str_df.pickle



----
# Download Job HTML

In [9]:

%run ../load_magic/storage.py
s = Storage()
base_url = 'https://www.indeed.com'
site_url = base_url + '/jobs'
site_url = '?'.join([site_url, 'q=data+scientist'])
site_url = '&'.join([site_url, 'jt=fulltime'])
site_url = '&'.join([site_url, 'remotejob=032b3046-06a3-4876-8dfd-474eb5e7ed11'])
site_url = '&'.join([site_url, 'vjk=ca16b63c03e40c57'])
#site_url = '&'.join([site_url, 'pp=gQAPAAABdY7RMKwAAAABkQdgZAAkAQBEKPpaoZstIag3f-UtQXXG_HFSo1gfBp9OQ0B0TvZ4yMp4AAA'])
start_num = 0
try:
    job_urls_list = s.load_object('job_urls_list')
except:
    job_urls_list = []
    s.store_objects(job_urls_list=job_urls_list)
space_regex = re.compile(r'[\s<>:"/\\\|\?\*_]+')
print_regex = re.compile(r'[\x9c-\x9d\uf0b7\u200b\ufb02]+')
s.encoding_type = ['latin1', 'iso8859-1', 'utf-8'][2]

In [21]:

fccid_htmls_list = [fn.split('_')[-1] for fn in os.listdir(saves_html_folder)]
row_count = len(job_urls_list)
for i, job_url in enumerate(job_urls_list):
    qs = urlparse(job_url).query
    query_dict = parse_qs(qs)
    fccid_str = query_dict['fccid'][0]
    file_name = f'{fccid_str}.html'
    if file_name not in fccid_htmls_list:
        job_page = requests.get(url=job_url)
        job_soup = BeautifulSoup(job_page.content, 'lxml')
        if not len(job_soup.text):
            break
        title_str = job_soup.find_all(name='title')[0].text.strip()
        clear_output(wait=True)
        print(f'{title_str}')
        print(f'{i}/{row_count}: {job_url}')
        if 'CAPTCHA' in title_str:
            break
        file_name = space_regex.sub('_', title_str)
        file_name = f'{file_name}_{fccid_str}.html'
        file_path = os.path.join(saves_html_folder, file_name)
        body_soup = job_soup.find_all(name='body')[0]
        html_str = '<html><head><title>' + title_str + '</title></head><body>'
        with open(file_path, 'w', encoding='utf-8') as f:
            print(html_str, file=f)
            for div_tag in body_soup.find_all(name='div', class_='jobsearch-JobComponent-description'):
                for s in div_tag.select('template'):
                    s.extract()
                for s in div_tag.select('script'):
                    s.extract()
                div_str = div_tag.prettify(formatter='html')
                div_str = print_regex.sub('', div_str)
                print(div_str, file=f)
            print('</body></html>', file=f)
print('Complete.')

Complete.


In [6]:

while start_num < 3000:
    page_url = '&'.join([site_url, f'start={start_num}'])
    start_num += 10
    site_page = requests.get(url=page_url)
    page_soup = BeautifulSoup(site_page.content, 'lxml')
    row_div_list = page_soup.find_all(name='div', class_=['row', 'result'])
    row_count = len(row_div_list)
    if row_count == 0:
        print('Nothing left')
        break
    for i, row_div in enumerate(row_div_list):
        link = row_div.find_all(name='a')[0]
        if 'title' in link.attrs:
            if 'href' in link.attrs:
                job_url = base_url + link['href']
                qs = urlparse(job_url).query
                query_dict = parse_qs(qs)
                if 'fccid' in query_dict:
                    job_urls_list.append(job_url)
                    s.store_objects(verbose=False, job_urls_list=job_urls_list)
                    clear_output(wait=True)
                    print(f'{page_url}')
                    print(f'{i}/{row_count}: {job_url}')

In [22]:

fccid_htmls_list = [fn.split('_')[-1] for fn in os.listdir(saves_html_folder)]
len(fccid_htmls_list)

273

In [22]:

import sys

command_str = f'{sys.executable} -m pip install --upgrade lxml'
print(command_str)
!{command_str}

C:\Users\dev\Anaconda3\envs\jh\python.exe -m pip install --upgrade lxml



----

In [2]:

import os

from selenium import webdriver
from base64 import b64encode
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import selenium.webdriver.support.ui as ui

In [4]:

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=site_url)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

In [5]:

saves_txt_folder = os.path.join(s.saves_folder, 'txt')
os.makedirs(name=saves_txt_folder, exist_ok=True)

In [6]:

row_div_list = sitePageSoup.find_all(name='div', class_=['row', 'result'])
reqs_regex = re.compile('ducat|xperience|equire')
html_regex = re.compile('<[^>]+>')

In [201]:

df.loc[0, 'navigable_parent']

'<div class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</div>'

In [ ]:

import pandas as pd

div_tag = div_tag_list[11]
child_str_list = get_navigable_children(div_tag, [])
df = pd.DataFrame(child_str_list, columns=['navigable_parent'])
df['is_header'] = False
#df.loc[[0, 2, 4, 17, 19], 'is_header'] = True
df

In [ ]:

text_list = ['What we are looking for', 'Key skills and Experience', 'Minimum qualifications', 'The Essentials', 'Qualifications',
             'Skills and experience', 'Required Qualifications', 'What your background looks like', "We(?:&rsquo;|')re looking for \w+ who have",
             'Experience/Minimum Qualifications', 'Requirements', 'We are looking for someone with', 'Qualifications']
reqs_regex = re.compile(f'^\\s*({'|'.join(text_list)}):?\\s*$', re.IGNORECASE | re.MULTILINE)
for match_obj in reqs_regex.finditer(text_str):
    # match start: match_obj.start()
    # match end (exclusive): match_obj.end()
    # matched text: match_obj.group()

In [39]:

[f'div_tag.{fn}' for fn in dir(div_tag) if 'child' in fn.lower()]

['div_tag._lastRecursiveChild', 'div_tag.childGenerator', 'div_tag.children', 'div_tag.findChild', 'div_tag.findChildren', 'div_tag.recursiveChildGenerator', 'div_tag.replaceWithChildren', 'div_tag.replace_with_children']

In [117]:

selector_list = ['body', 'div', 'div.jobsearch-ViewJobLayout-fluidContainer',
                 'div.jobsearch-ViewJobLayout-content.jobsearch-ViewJobLayout-mainContent', 'div', 'div',
                 'div.jobsearch-ViewJobLayout-jobDisplay', 'div.jobsearch-JobComponent', 'div.jobsearch-JobComponent-description']
content_selector = ' > '.join(selector_list)

def has_role_attr(tag):
    
    return tag.has_attr('role')

In [ ]:

def has_class_and_id(tag):
    
    return tag.has_attr('class') and tag.has_attr('id')

# <span class="summary">We need a <b>data</b> <b>scientist</b> and <b>data</b> wrangler. Maybe survey <b>data</b>. 
# Better still if you have some demonstrable experience with more advanced machine learning methods...</span>
summary_list = sitePageSoup.find_all(name='span', class_='summary')
for summary in summary_list:
    print(summary.text.strip())
    print()

In [ ]:

# <a target="_blank" id="sja5" data-tn-element="jobTitle" class="jobtitle turnstileLink" 
# href="https://www.indeed.com/pagead/clk?mo=r&amp;ad=-...-...-...-...-...-...-...&
# amp;p=5&amp;sk=&amp;fvj=1&amp;tk=1c06s8995av53coj&amp;jsa=6565" 
# title="Perception Scientist for Marine Autonomy" rel="noopener nofollow" 
# onmousedown="sjomd('sja5'); clk('sja5');" onclick="setRefineByCookie([]); sjoc('sja5',0); convCtr('SJ')"
# >Perception <b>Scientist</b> for Marine Autonomy</a>
siteCss = '#sja5'
siteLinks = sitePageSoup.select(siteCss)
print(siteLinks)

max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0]["href"].split('/')[-1].split('-')[1].split(',')[0])
print(max_page)

In [6]:
#<h1 class="srp-header">25,589 Used Vehicles for sale</h1>

for parser in ['lxml', 'html5lib', 'html.parser']:
    sitePageSoup = BeautifulSoup(page_html, parser)
    print(parser, len(sitePageSoup.select('h1.srp-header')))

#max_page = int(sitePageSoup.find_all("a", class_="js-last-page")[0].text)
#print(max_page)

lxml 1
html5lib 0
html.parser 1
